In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [3]:
%pip install "tensorflow==2.16.1"

   ---------------------------------------- 0.0/377.1 MB ? eta -:--:--
   ---------------------------------------- 4.5/377.1 MB 24.4 MB/s eta 0:00:16
    --------------------------------------- 6.6/377.1 MB 16.1 MB/s eta 0:00:24
    --------------------------------------- 8.4/377.1 MB 14.1 MB/s eta 0:00:27
   - -------------------------------------- 10.0/377.1 MB 11.7 MB/s eta 0:00:32
   - -------------------------------------- 10.7/377.1 MB 10.3 MB/s eta 0:00:36
   - -------------------------------------- 13.6/377.1 MB 10.8 MB/s eta 0:00:34
   - -------------------------------------- 14.9/377.1 MB 10.2 MB/s eta 0:00:36
   - -------------------------------------- 18.1/377.1 MB 10.9 MB/s eta 0:00:34
   -- ------------------------------------- 19.9/377.1 MB 10.6 MB/s eta 0:00:34
   -- ------------------------------------- 21.8/377.1 MB 10.5 MB/s eta 0:00:34
   -- ------------------------------------- 23.6/377.1 MB 10.3 MB/s eta 0:00:35
   -- ------------------------------------- 26.0/377

In [4]:
from tensorflow import keras

In [7]:
p_train = r"C:\Users\TSJ20\xwechat_files\wxid_c8919vr6p4qg22_429f\msg\file\2025-10\scale_train_data.csv"
p_test  = r"C:\Users\TSJ20\xwechat_files\wxid_c8919vr6p4qg22_429f\msg\file\2025-10\scale_test_data.csv"
train = pd.read_csv(p_train)
test  = pd.read_csv(p_test)

In [8]:
target = 'NSP' if 'NSP' in train.columns else 'CLASS'

In [20]:
y_train = train[target].to_numpy(dtype=int)
X_train = train.loc[:, train.columns != target].to_numpy(dtype='float32')

y_test  = test[target].to_numpy(dtype=int)
X_test  = test.loc[:,  test.columns  != target].to_numpy(dtype='float32')

In [22]:
y_train = y_train - 1
y_test  = y_test  - 1

In [24]:
vals, cnts = np.unique(y_train, return_counts=True)
class_weight = {int(v): float(cnts.max()/c) for v,c in zip(vals, cnts)}
print("class_weight:", class_weight)

class_weight: {0: 1.0, 1: 5.974226804123711, 2: 8.585185185185185}


In [28]:
num_classes = int(np.unique(y_train).size)
print("num_classes =", num_classes, " labels in train:", np.unique(y_train))

num_classes = 3  labels in train: [0 1 2]


In [30]:
inputs = keras.Input(shape=(X_train.shape[1],))
x = keras.layers.BatchNormalization()(inputs)
x = keras.layers.Dense(128, activation="relu")(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(64, activation="relu")(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

In [32]:
es = keras.callbacks.EarlyStopping(
    monitor="val_accuracy", patience=10, restore_best_weights=True
)
hist = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=200, batch_size=32,
    class_weight=class_weight,
    callbacks=[es], verbose=1
)


Epoch 1/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.4882 - loss: 2.1512 - val_accuracy: 0.1275 - val_loss: 2.5602
Epoch 2/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8706 - loss: 1.3268 - val_accuracy: 0.1309 - val_loss: 2.7095
Epoch 3/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8445 - loss: 1.1088 - val_accuracy: 0.3121 - val_loss: 1.5427
Epoch 4/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9025 - loss: 0.9116 - val_accuracy: 0.4161 - val_loss: 1.3846
Epoch 5/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9109 - loss: 0.8831 - val_accuracy: 0.6242 - val_loss: 0.8527
Epoch 6/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9059 - loss: 0.8138 - val_accuracy: 0.7752 - val_loss: 0.4331
Epoch 7/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9076 - loss: 0.7760 - val_accuracy: 0.8356 - val_loss: 0.3443
Epoch 8/200
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9202 - loss: 0.7172 - val_accuracy: 0.9027 - v

In [34]:
proba = model.predict(X_test)
pred = proba.argmax(axis=1)

print("Accuracy:", accuracy_score(y_test, pred))
print(classification_report(y_test, pred, digits=2))
print("Confusion matrix:\n", confusion_matrix(y_test, pred))

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
Accuracy: 0.9310344827586207
              precision    recall  f1-score   support

           0       0.99      0.96      0.97       496
           1       0.85      0.82      0.83       101
           2       0.60      0.90      0.72        41

    accuracy                           0.93       638
   macro avg       0.81      0.89      0.84       638
weighted avg       0.94      0.93      0.93       638

Confusion matrix:
 [[474  12  10]
 [  3  83  15]
 [  1   3  37]]
